In [1]:
import random, copy
from pprint import pprint

class JadwalKuliah:
    def __init__(
            self, 
            kode_matkul, kode_dosen, sks_akademik, kode_ruangan, kapasitas, 
            hari, jam_mulai, jam_selesai, 
            tipe_kelas, program_studi):
        self.kode_matkul = kode_matkul
        self.kode_dosen = kode_dosen
        self.sks_akademik = sks_akademik
        self.kode_ruangan = kode_ruangan
        self.kapasitas = kapasitas
        self.hari = hari
        self.jam_mulai = jam_mulai
        self.jam_selesai = jam_selesai
        self.tipe_kelas = tipe_kelas  # 'TEORI' atau 'PRAKTIKUM'
        self.program_studi = program_studi

    def __repr__(self):
        return f"<{self.kode_matkul}, {self.kode_dosen}, {self.sks_akademik}, {self.kode_ruangan}, {self.hari}, {self.jam_mulai}, {self.jam_selesai}, {self.tipe_kelas}>\n"

# print([i for i in range(9, 13)])
# [9, 10, 11, 12]

BOBOT_PENALTI = {
    "dosen_bentrok": 15, # dosen mengajar 2/lebih kelas bersamaan
    "ruangan_bentrok": 15, # kelas digunakan 2/lebih mata kuliah bersamaan
    "asdos_nabrak_dosen": 15, # kelas dosen dan asistensi berjalan dalam waktu yang sama
    "kelas_dosen_tidak_ditemukan": 15,
    "dosen_terbebani": 10, # beban sks melebihi batas
    "kapasitas_kelas_terbatas": 10, # kalau jumlah kapasitas kelas < jumlah mahasiswa
    "weekend_class": 10, # kelas diadakan di hari Sabtu
    "istirahat": 5, # kelas diadakan di jam istirahat
    "salah_tipe_ruangan": 3, # tipe ruangan salah (PRAKTIKUM di TEORI)
    "diluar_jam_kerja": 10, # as it says
    "melanggar_preferensi": 5 # as it says
}

# List mata kuliah ( + jumlah mahasiswa )
# {
#     'kode': str, 
#     'nama': str, 
#     'kelompok': str, 
#     'sks_akademik': int, 
#     'sks_bayar': int, 
#     'prodi': str, 
#     'tipe_kelas': 'TEORI' or 'PRAKTIKUM', 
#     'bidang': list, 
#       "asistensi": bool, 
#       "tipe_kelas_asistensi": "TEORI" or 'PRAKTIKUM',
#       'integrated_class': bool,
#     'jumlah_mahasiswa': int
#   }
matakuliah_list = [
  # { "kode": "DS001", "nama": "GATAU NI APAAN", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 PROMAG", "tipe_kelas": "TEORI", "bidang": ["RANDOM"], "asistensi": True, "tipe_kelas_asistensi": "TEORI", "jumlah_mahasiswa": 300},

  # tahun 1
  { "kode": "TC511", "nama": "MATEMATIKA DISKRIT", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ["MATHEMATICS"], "asistensi": True, "tipe_kelas_asistensi": "TEORI", "jumlah_mahasiswa": 300},
  { "kode": "TC512", "nama": "PENGANTAR TEKNOLOGI INFORMASI", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ['SOFTWARE'], "asistensi": True, "tipe_kelas_asistensi": "PRAKTIKUM", "jumlah_mahasiswa": 300},
  { "kode": "TC513", "nama": "DASAR-DASAR PEMROGRAMAN", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": ['SOFTWARE'], "asistensi": True, "tipe_kelas_asistensi": "PRAKTIKUM", "jumlah_mahasiswa": 300},
  { "kode": "TC514", "nama": "BAHASA INGGRIS", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], "jumlah_mahasiswa": 300},
  { "kode": "TC515", "nama": "SISTEM BASIS DATA", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ['DATA'], "asistensi": True, "tipe_kelas_asistensi": "PRAKTIKUM", "jumlah_mahasiswa": 300},

  # tahun 2
  { "kode": "TC611", "nama": "KECERDASAN BUATAN", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ['DATA'], "asistensi": True, "tipe_kelas_asistensi": "TEORI", "jumlah_mahasiswa": 300 },
  { "kode": "TC612", "nama": "PEMROGRAMAN WEB", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": ['SOFTWARE'], "asistensi": True, "tipe_kelas_asistensi": "PRAKTIKUM", "integrated_class": True, "jumlah_mahasiswa": 300 },
  { "kode": "TC613", "nama": "PEMROGRAMAN BERORIENTASI OBJEK", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": ['SOFTWARE'], "asistensi": True, "tipe_kelas_asistensi": "PRAKTIKUM", "integrated_class": True, "jumlah_mahasiswa": 300 },
  { "kode": "TC614", "nama": "DASAR PEMROGRAMAN JARINGAN", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ['NETWORK'], "jumlah_mahasiswa": 300 },
  { "kode": "TC615", "nama": "ORGANISASI DAN ARSITEKTUR KOMPUTER", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ['NETWORK'], "jumlah_mahasiswa": 300 },

  # tahun 3 - SE
  { "kode": "TC712", "nama": "PEMROGRAMAN MOBILE", "kelompok": "BIDANG MINAT SOFTWARE ENGINEERING", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": ['SOFTWARE'], 'jumlah_mahasiswa': 130 },
  { "kode": "TC715", "nama": "MANAJEMEN PROYEK", "kelompok": "BIDANG MINAT SOFTWARE ENGINEERING", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ['SOFTWARE'], 'jumlah_mahasiswa': 130 },
  { "kode": "TC717", "nama": "PENGUJIAN SISTEM", "kelompok": "BIDANG MINAT SOFTWARE ENGINEERING", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ['SOFTWARE'], 'jumlah_mahasiswa': 130 },
  { "kode": "TC719", "nama": "PEMROGRAMAN BERORIENTASI PLATFORM", "kelompok": "BIDANG MINAT SOFTWARE ENGINEERING", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": ['SOFTWARE'], 'jumlah_mahasiswa': 130 },
  { "kode": "TC781", "nama": "SERTIFIKASI SOFTWARE ENGINEERING - ORACLE", "kelompok": "BIDANG MINAT SOFTWARE ENGINEERING", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ['SOFTWARE'], 'jumlah_mahasiswa': 130 },

  # tahun 3 - network
  { "kode": "TC751", "nama": "PERANCANGAN DAN MANAJEMEN JARINGAN", "kelompok": "BIDANG MINAT NETWORK ENGINEERING", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ["NETWORK"], 'jumlah_mahasiswa': 30 },
  { "kode": "TC754", "nama": "ROUTING DAN SWITCHING 1", "kelompok": "BIDANG MINAT NETWORK ENGINEERING", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": ["NETWORK"], 'jumlah_mahasiswa': 30 },
  { "kode": "TC757", "nama": "KEAMANAN JARINGAN", "kelompok": "BIDANG MINAT NETWORK ENGINEERING", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ["NETWORK"], 'jumlah_mahasiswa': 30 },
  { "kode": "TC783", "nama": "SERTIFIKASI NETWORK ENGINEERING - RSNA", "kelompok": "BIDANG MINAT NETWORK ENGINEERING", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ["NETWORK"], 'jumlah_mahasiswa': 30 },

  # tahun 3 - DS
  { "kode": "TC733", "nama": "DEEP LEARNING", "kelompok": "BIDANG MINAT DATA SCIENCE", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ["DATA"], 'jumlah_mahasiswa': 120 },
  { "kode": "TC734", "nama": "PEMODELAN SPASIAL", "kelompok": "BIDANG MINAT DATA SCIENCE", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ["DATA"], 'jumlah_mahasiswa': 120 },
  { "kode": "TC737", "nama": "PEMROSESAN BAHASA ALAMI", "kelompok": "BIDANG MINAT DATA SCIENCE", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ["DATA"], 'jumlah_mahasiswa': 120 },
  { "kode": "TC782", "nama": "SERTIFIKASI DATA SCIENCE - ORACLE", "kelompok": "BIDANG MINAT DATA SCIENCE", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": ["DATA"], 'jumlah_mahasiswa': 120 },

  # # tahun 4
  # { "kode": "TC811","nama": "TUGAS TALENTA UNGGUL 1", "kelompok": "TALENTA UNGGUL", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
  # { "kode": "TC812","nama": "TUGAS TALENTA UNGGUL 2", "kelompok": "TALENTA UNGGUL", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
  # { "kode": "TC813","nama": "PUBLIKASI KARYA ILMIAH", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
  # { "kode": "TC821", "nama": "KERJA PRAKTEK", "kelompok": "TALENTA UNGGUL", "sks_akademik": 8, "sks_bayar": 8, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
  # { "kode": "TC822", "nama": "LAPORAN KERJA PRAKTEK", "kelompok": "TALENTA UNGGUL", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },

  # { "kode": "TC833", "nama": "PEMBIMBINGAN INDUSTRI 3", "kelompok": "MERDEKA BELAJAR KAMPUS MERDEKA (MBKM)", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
  # { "kode": "TC841", "nama": "MBKM 1", "kelompok": "MERDEKA BELAJAR KAMPUS MERDEKA (MBKM)", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
  # { "kode": "TC842", "nama": "MBKM 2", "kelompok": "MERDEKA BELAJAR KAMPUS MERDEKA (MBKM)", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
]

# List dosen
dosen_list = [
  # { "nip": "99001", "nama": "KEN", "prodi": "S1 PROMAG", "status": "DOSEN TETAP", "kepakaran": ["RANDOM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },

  { "nip": "67001", "nama": "ALZ", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['DATA', "MATHEMATICS"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67002", "nama": "CHRISTINE", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['DATA', "INTELLIGENT SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67003", "nama": "DIAN", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['NETWORK', "NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67004", "nama": "EKO", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['SOFTWARE', "INFORMATION TECHNOLOGY"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67005", "nama": "EVANGS", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['DATA', "INTELLIGENT SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67006", "nama": "HENDRY", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['SOFTWARE', "INTELLIGENT SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67007", "nama": "HINDRIYANTO", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['NETWORK', "INTELLIGENT SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67008", "nama": "INDRASTANTI", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['NETWORK', "NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67009", "nama": "ADI", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['DATA'], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67010", "nama": "INEKE", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['MATHEMATICS', 'DATA'], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67011", "nama": "NINA", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['SOFTWARE', "SOFTWARE ENGINEERING"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "68001", "nama": "OCSA", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['SOFTWARE'], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67012", "nama": "RADIUS", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['SOFTWARE', "SOFTWARE ENGINEERING"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67013", "nama": "YULIANTO", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['SOFTWARE', "INFORMATION SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67014", "nama": "SUPRIHADI", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['DATA', "DIGITAL BUSINESS"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67015", "nama": "TEGUH", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['NETWORK', "NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67016", "nama": "THEOPHILUS", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['NETWORK', "NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67017", "nama": "WIWIN", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['NETWORK', "NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67018", "nama": "YEREMIA", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['SOFTWARE', "INFORMATION SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67019", "nama": "YOS", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['SOFTWARE', "SOFTWARE ENGINEERING"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "56001", "nama": "BUDHI", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['NETWORK', "INFORMATION SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67020", "nama": "SUHARYADI", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['DATA'], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "55001", "nama": "ARIYA", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['DATA'], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67021", "nama": "RICHARD", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ['SOFTWARE'], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67022", "nama": "DANIEL", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ["INFORMATION TECHNOLOGY"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67023", "nama": "IRWAN", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ['NETWORK', "NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67024", "nama": "YESSICA", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ["MATHEMATICS"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67025", "nama": "ADE", "prodi": "", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67026", "nama": "KRISTOKO", "prodi": "", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67027", "nama": "FELIX", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ['SOFTWARE', "SOFTWARE ENGINEERING"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67028", "nama": "ARIE", "prodi": "", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67029", "nama": "ERWIEN", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ['NETWORK', "NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "55002", "nama": "WINARSO", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ['SOFTWARE', "INFORMATION TECHNOLOGY"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67030", "nama": "TEGUH", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ['NETWORK', "DIGITAL BUSINESS"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67031", "nama": "RISSAL", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ['NETWORK', "NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67032", "nama": "KRISMIYATI", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ["EDUCATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67033", "nama": "HANITA", "prodi": "", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67034", "nama": "CHARITAS", "prodi": "", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67035", "nama": "FAJAR", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67036", "nama": "RIKO", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": ['NETWORK'], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67037", "nama": "CHERYL", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67038", "nama": "DANANG", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67039", "nama": "EDNA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67040", "nama": "HILDA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67041", "nama": "MARTZA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": ['NETWORK'], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "RAYNALDO", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "68002", "nama": "VIKKY", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67042", "nama": "VYORBIGGER", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": ['SOFTWARE'], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67043", "nama": "ERYKA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": ['MATHEMATICS'], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "68003", "nama": "NATALIA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": ['MATHEMATICS'], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "WILSON", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "YULIA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "CHRISTA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "SEPTOVAN", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67044", "nama": "ANGLING", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "11001", "nama": "DANIEL", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "AFIYATAR", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "VIRGELIUS", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } }
]

# List ruang (TEORI = T, PRAKTIKUM = P)
# {#     "kode": "", #     "tipe_ruangan": "", #     "kapasitas": 0, #     "plot": []# }
ruang_list = [
  { "kode": "FTI300", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["GENERAL"] },
  { "kode": "FTI301", "tipe_ruangan": "TEORI", "kapasitas": 19, "plot": ["S1 PERPUSTAKAAN DAN SAINS INFORMASI"] },
  { "kode": "FTI302", "tipe_ruangan": "TEORI", "kapasitas": 21, "plot": ["S1 HUBUNGAN MASYARAKAT"] },
  { "kode": "FTI306", "tipe_ruangan": "TEORI", "kapasitas": 43, "plot": ["GENERAL"] },
  { "kode": "FTI308", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["GENERAL"] },
  { "kode": "FTI329", "tipe_ruangan": "TEORI", "kapasitas": 44, "plot": ["GENERAL"] },
  { "kode": "FTI331", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["GENERAL"] },
  { "kode": "FTI333", "tipe_ruangan": "TEORI", "kapasitas": 56, "plot": ["GENERAL"] },
  { "kode": "FTI401", "tipe_ruangan": "TEORI", "kapasitas": 25, "plot": ["S2 MAGISTER SISTEM INFORMASI"] },
  { "kode": "FTI420", "tipe_ruangan": "TEORI", "kapasitas": 39, "plot": ["GENERAL"] },
  { "kode": "FTI422", "tipe_ruangan": "TEORI", "kapasitas": 45, "plot": ["GENERAL"] },
  { "kode": "FTI424", "tipe_ruangan": "TEORI", "kapasitas": 44, "plot": ["GENERAL"] },
  { "kode": "FTI504", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["S1 PENDIDIKAN TEKNIK INFORMATIKA DAN KOMPUTER"] },
  { "kode": "FTI505", "tipe_ruangan": "TEORI", "kapasitas": 21, "plot": ["GENERAL"] },
  { "kode": "FTI508", "tipe_ruangan": "TEORI", "kapasitas": 33, "plot": ["GENERAL"] },
  { "kode": "FTI511", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["GENERAL"] },

  { "kode": "FTI310", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 32, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI402", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 18, "plot": ["S1 SIA"], "specs": [] },
  { "kode": "FTI429", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 3, "plot": ["S1 DESAIN KOMUNIKASI VISUAL"], "specs": [] },
  { "kode": "FTI431", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 20, "plot": ["NETWORK", "CISCO"], "specs": [] },
  { "kode": "FTI451", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 27, "plot": ["SINARMAS"], "specs": [] },
  { "kode": "FTI453", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 7, "plot": ["D3 TEKNIK INFORMATIKA", "HARDWARE"], "specs": [] },
  { "kode": "FTI454", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 20, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI455", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 41, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI457", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI459", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI461", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["D3 TEKNIK INFORMATIKA"], "specs": [] },
  { "kode": "FTI463", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI464", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI465", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI467", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI469", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI506", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 30, "plot": ["ORACLE"], "specs": [] },
  { "kode": "FTI507", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 20, "plot": ["NETWORK", "CISCO"], "specs": [] },
  { "kode": "FTI400", "tipe_ruangan": "RAPAT", "kapasitas": 103, "plot": ["GENERAL"] }
]

# # testing
# pilihan_waktu = [ i for i in range(7, 19)] # 7-18
# dosen_s1_ti = [ d for d in dosen_list if d["prodi"] == "S1 TEKNIK INFORMATIKA"]
# bobot_dosen_pakar = [((len(set(d['kepakaran']) & set([]))*5) or 1) for d in dosen_s1_ti]
# dosen_pakar_s1_ti = [ d for d in dosen_s1_ti if len(set(d['kepakaran']) & set(['mathematics'.upper()])) > 0]
# ruangan_general_s1_ti = [r for r in ruang_list if "S1 TEKNIK INFORMATIKA" in r["plot"] or "GENERAL" in r["plot"]]
# ruangan_general_s1_ti_teori = [r for r in ruang_list if ("S1 TEKNIK INFORMATIKA" in r["plot"] or "GENERAL" in r["plot"]) and r['tipe_ruangan'] == "TEORI"]

# print(random.choices(population=dosen_s1_ti, weights=bobot_dosen_pakar, k=1)[0])

In [2]:
import xlsxwriter
from IPython.display import FileLink
import os
from collections import defaultdict

def export_jadwal_to_excel(jadwal_list, filename="jadwal_kuliah.xlsx"):
    fixed_headers = [
        'kode_matkul', 'nama_matkul',
        'kode_dosen', 'nama_dosen', 'sks_akademik', 
        'kode_ruangan', 'kapasitas', 
        'hari', 'jam_mulai', 'jam_selesai',
        'tipe_kelas'
    ]

    # 🔹 Group berdasarkan program_studi
    grouped = defaultdict(list)
    for jadwal in jadwal_list:
        grouped[jadwal.program_studi].append(jadwal)

    # 🔹 Buat workbook
    workbook = xlsxwriter.Workbook(filename)

    # Format header
    format_header = workbook.add_format({
        'align': 'center',
        'bg_color': '#99CCFF',
        'valign': 'vcenter',
        'bold': True
    })
    format_as = workbook.add_format({
        'align': 'center',
        'bg_color': '#b9ebc6',
    })

    # Loop setiap group dan tulis ke sheet terpisah
    for program_studi in sorted(grouped.keys()):
        sheet_name = program_studi[:31]  # Sheet name max 31 chars
        worksheet = workbook.add_worksheet(sheet_name)

        row_idx = 0
        col_widths = [len(h) for h in fixed_headers]

        # Tulis header kolom
        for col_idx, header in enumerate(fixed_headers):
            worksheet.write(row_idx, col_idx, header.replace('_', ' '), format_header)
        row_idx += 1

        # Sort dan tulis data
        sorted_group = sorted(grouped[program_studi], key=lambda x: x.kode_matkul)
        for jadwal in sorted_group:
            for col_idx, attr in enumerate(fixed_headers):
                if attr == "nama_matkul":
                    value = getattr(jadwal, 'kode_matkul')
                    value = next((m['nama'] for m in matakuliah_list if m['kode'] == value[:-1] or m['kode'] == value[:-4]), None)
                elif attr == "nama_dosen":
                    value = getattr(jadwal, 'kode_dosen')
                    value = next((d['nama'] for d in dosen_list if d['nip'] == value), None)
                elif attr == "kapasitas":
                    value = getattr(jadwal, "kode_matkul")
                    if value[-2:] == "AS":
                        kapasitas_dosen = next((sesi.kapasitas for sesi in jadwal_list if sesi.kode_matkul == value[:-3]), None)
                        value = kapasitas_dosen
                    else:
                        value = getattr(jadwal, attr)
                else:
                    value = getattr(jadwal, attr)

                worksheet.write(row_idx, col_idx, value, format_as if getattr(jadwal, 'kode_dosen') == "AS" else None)

                val_len = len(str(value))
                if val_len > col_widths[col_idx]:
                    col_widths[col_idx] = val_len
            row_idx += 1

        # Set lebar kolom
        for col_idx, width in enumerate(col_widths):
            worksheet.set_column(col_idx, col_idx, width + 2)

    # Simpan workbook
    workbook.close()
    print(f"✅ File '{filename}' berhasil dibuat.")
    print('Excel disimpan di:', os.getcwd())
    return FileLink(filename)

In [3]:
def find_available_schedule(jadwal:list, kode:str, hari:str):
    """
    Mencari waktu yang tersedia dari jadwal yang sudah ada berdasarkan hari yang dipilih.

    Args:
        jadwal (list): List jadwal yang sudah ada.
        kode (str): Kode yang akan diperiksa (kode ruangan / nip).
        hari (str): Hari yang akan diperiksa jadwalnya.

    Returns:
        list: List jadwal yang bisa digunakan (jam).
    """
    
    pilihan_jam = list(range(7, 19)) if hari != "SABTU" else list(range(7, 13))
    
    jadwal_sesuai_hari = [sesi for sesi in jadwal[kode] if sesi['hari'] == hari] if jadwal[kode] else []
    used_jam = []
    for sesi in jadwal_sesuai_hari:
        used_jam.extend(range(sesi['jam_mulai'], sesi['jam_selesai'] + 1))

    return [h for h in pilihan_jam if h not in used_jam]

def angka_ke_huruf(n: int):
    """
    Generate alphabet (A-Z) dari angka (n) yang diberikan.

    Args:
        n (int): Angka yang ingin diparsing ke alphabet.

    Returns:
        chr|None: Alphabet yang sesuai dengan angka yang diberikan.
    """

    if 1 <= int(n) <= 26:
        return chr(64 + n)  # Karena ord('A') = 65
    else:
        return None  # Diluar jangkauan 1-26

In [4]:
'''
    Question:
'''

def repair_jadwal(jadwal, matakuliah_list, dosen_list, ruang_list):
    pilihan_hari_dosen = ["SENIN", "SELASA", "RABU", "KAMIS", "JUMAT"]
    pilihan_hari_asisten = copy.deepcopy(pilihan_hari_dosen)
    pilihan_hari_asisten.append("SABTU")
    pilihan_jam = list(range(7, 20))

    max_attempt = 10

    jadwal_dosen = {}
    jadwal_ruangan = {}
    beban_dosen = {}

    # AMBIL INITIAL
    for ruang in ruang_list:
        if ruang['kode'] not in jadwal_ruangan: jadwal_ruangan[ruang['kode']] = []
    for dosen in dosen_list:
        if dosen['nip'] not in jadwal_dosen: jadwal_dosen[dosen['nip']] = []
        if dosen['nip'] not in beban_dosen: beban_dosen[dosen['nip']] = 0

    # HAPUS DATA MATKUL DUPLIKAT
    unique_course = set()
    filtered_jadwal = []

    for sesi in jadwal:
        if sesi.kode_matkul not in unique_course:
            unique_course.add(sesi.kode_matkul)
            filtered_jadwal.append(sesi)

    # Hasil jadwal tanpa course duplikat
    jadwal = filtered_jadwal

    # LOOP PERTAMA:
    # 1. REPAIR JADWAL DOSEN KALAU CONFLICT DENGAN JADWAL LAIN (DONE)
    # 2. KALAU MATKUL ADA PRAKTIKUM, CEK EKSISTENSI KELAS ASISTENNYA (DONE)
    # 2a.KALAU KELAS ASISTENNYA NOT EXIST, CIPTAKAN SCR RANDOM (DONE)
    for sesi in jadwal:
        if sesi.kode_dosen != "AS":
            if sesi.kode_dosen not in jadwal_dosen: jadwal_dosen[sesi.kode_dosen] = []
            matkul = next((m for m in matakuliah_list if m['kode'] == sesi.kode_matkul[:-1]), None)
            dosen = next((d for d in dosen_list if d['nip'] == sesi.kode_dosen), None)
            preferensi_hari_dosen = [d for d in pilihan_hari_dosen if d not in dosen['preferensi']['hindari_hari']]

            if matkul:
                conflict = False

                for sesi_lain in jadwal_ruangan[sesi.kode_ruangan]:
                    if sesi.hari == sesi_lain['hari']:
                        if sesi_lain['jam_mulai'] <= sesi.jam_mulai < sesi_lain['jam_selesai'] or sesi_lain['jam_mulai'] < sesi.jam_selesai <= sesi_lain['jam_selesai']:
                            conflict = True
                            break

                for sesi_lain in jadwal_dosen[sesi.kode_dosen]:
                    if sesi.hari == sesi_lain['hari']:
                        if sesi_lain['jam_mulai'] <= sesi.jam_mulai < sesi_lain['jam_selesai'] or sesi_lain['jam_mulai'] < sesi.jam_selesai <= sesi_lain['jam_selesai']:
                            conflict = True
                            break

                attempt = 1
                excluded_day = []
                excluded_room = []
                # 1. Repair Jadwal Dosen
                while conflict and attempt <= max_attempt:
                    # print(sesi.kode_matkul, sesi.kode_dosen, sesi.sks_akademik, sesi.hari)
                    available_room_schedule = find_available_schedule(jadwal_ruangan, sesi.kode_ruangan, sesi.hari)
                    available_lecturer_schedule = find_available_schedule(jadwal_dosen, sesi.kode_dosen, sesi.hari)
                    # print('room     ', available_room_schedule)
                    # print('lecture  ', available_lecturer_schedule)
                    
                    available_room_x_lecturer = list(set(available_room_schedule) & set(available_lecturer_schedule))
                    
                    status = False
                    for jam in available_room_x_lecturer:
                        rentang_waktu = list(range(jam, jam + sesi.sks_akademik + 1))
                        if all(r in available_room_x_lecturer for r in rentang_waktu) and jam != 12:
                            status = True
                            conflict = False
                            sesi.jam_mulai = jam
                            sesi.jam_selesai = jam + sesi.sks_akademik
                            break
                        else:
                            status = False

                    if not status:
                        excluded_day.append(sesi.hari)
                        # print('pil ', pilihan_hari_dosen)
                        # print('exc ', excluded_day)
                        # print('      ', [d for d in pilihan_hari_dosen if d not in excluded_day])
                        # print('      ', available_room_x_lecturer)
                        if all(d in excluded_day for d in preferensi_hari_dosen):
                            excluded_room.append(sesi.kode_ruangan)
                            calon_ruang_pengganti = [
                                r for r in ruang_list if 
                                    (matkul['prodi'] in r['plot'] or 'GENERAL' in r['plot']) and
                                    r['kapasitas'] >= 35 and
                                    (
                                        r['tipe_ruangan'] == sesi.tipe_kelas or 
                                        (r['tipe_ruangan'] == "RAPAT" if not matkul.get('asistensi') else False)
                                    ) and r['kode'] not in excluded_room
                            ]
                            if not calon_ruang_pengganti: print(sesi.kode_matkul, excluded_room, calon_ruang_pengganti)
                            bobot_calon_ruang_pengganti = [
                                (len(set(r['plot']) & set(matkul['bidang']))*10 or 1)
                                for r in calon_ruang_pengganti
                            ]
                            ruang_pengganti = random.choices(
                                population=calon_ruang_pengganti, 
                                weights=bobot_calon_ruang_pengganti, 
                                k=1)[0]
                            sesi.kode_ruangan = ruang_pengganti['kode']
                        else:
                            sesi.hari = random.choice([d for d in preferensi_hari_dosen if d not in excluded_day])

                    attempt += 1
                
                jadwal_ruangan[sesi.kode_ruangan].append({'hari': sesi.hari, 'jam_mulai': sesi.jam_mulai, 'jam_selesai': sesi.jam_selesai})
                jadwal_dosen[sesi.kode_dosen].append({'hari': sesi.hari, 'jam_mulai': sesi.jam_mulai, 'jam_selesai': sesi.jam_selesai})

                # 2. Cek eksistensi kelas asisten jika memang ada
                if matkul.get('asistensi') and not matkul.get('integrated_class'):
                    kelas_asisten = next((j for j in jadwal if j.kode_matkul == sesi.kode_matkul + '-AS'), None)
                    # 2a. ciptakan secara random jika not exist
                    if not kelas_asisten:
                        calon_ruang_asisten = [
                            r for r in ruang_list if 
                                (matkul['prodi'] in r['plot'] or 'GENERAL' in r['plot']) and
                                r['kapasitas'] >= sesi.kapasitas and
                                r['tipe_ruangan'] == matkul['tipe_kelas_asistensi']
                        ]
                        bobot_calon_ruang_asisten = [
                            (len(set(r['plot']) & set(matkul['bidang']))*10 or 1)
                            for r in calon_ruang_asisten
                        ]
                        ruang_asisten = random.choices(
                            population=calon_ruang_asisten, 
                            weights=bobot_calon_ruang_asisten, 
                            k=1)[0]
                        
                        hari_asisten = random.choice(pilihan_hari_asisten)
                        jam_mulai_asisten = random.choice([h for h in pilihan_jam if h + sesi.sks_akademik <= max(pilihan_jam)])
                        jam_selesai_asisten = jam_mulai_asisten + sesi.sks_akademik

                        sesi_asisten = JadwalKuliah(
                            kapasitas    = ruang_asisten['kapasitas'],
                            kode_matkul  = f"{sesi.kode_matkul}-AS",
                            kode_dosen   = "AS",
                            sks_akademik = matkul['sks_akademik'],
                            kode_ruangan = ruang_asisten['kode'],
                            hari         = hari_asisten,
                            jam_mulai    = jam_mulai_asisten,
                            jam_selesai  = jam_selesai_asisten,
                            tipe_kelas   = matkul['tipe_kelas_asistensi'],
                            program_studi= matkul['prodi']
                        )
                        jadwal.append(sesi_asisten)
    
    # LOOP KEDUA:
    # 1. CEK EKSISTENSI KELAS DOSENNYA (DONE)
    # 1a.KALAU KELAS DOSENNYA NOT EXIST, CIPTAKAN DENGAN METODE YANG BENAR
    # 2. REPAIR JADWAL ASISTEN KALAU BENTROK DENGAN KELAS LAIN
    for sesi in jadwal:
        if sesi.kode_dosen == "AS": # KODE ASISTEN
            matkul = next((m for m in matakuliah_list if m['kode'] == sesi.kode_matkul[:-4]), None)

            if matkul:
                # 1. Cek eksistensi kelas dosennya
                kelas_dosen = next((j for j in jadwal if j.kode_matkul == sesi.kode_matkul[:-3]), None)

                if not kelas_dosen:
                    calon_ruang_dosen = [
                        r for r in ruang_list if 
                            (matkul['prodi'] in r['plot'] or 'GENERAL' in r['plot']) and
                            r['kapasitas'] <= sesi.kapasitas and
                            (
                                r['tipe_ruangan'] == matkul['tipe_kelas'] or 
                                (r['tipe_ruangan'] == "RAPAT" if not matkul.get('asistensi') else False)
                            )
                    ]
                    bobot_calon_ruang_dosen = [
                        (len(set(r['plot']) & set(matkul['bidang']))*10 or 1)
                        for r in calon_ruang_dosen
                    ]
                    ruang_dosen = random.choices(
                        population=calon_ruang_dosen, 
                        weights=bobot_calon_ruang_dosen, 
                        k=1)[0]
                    
                    calon_dosen = [d for d in dosen_list if d["prodi"] == matkul['prodi']]
                    bobot_calon_dosen = [len(set(d['kepakaran']) & set(matkul['bidang']))*10 - (beban_dosen[d['nip']]) for d in calon_dosen]
                    dosen_pengajar = random.choices(
                        population=calon_dosen,
                        weights=bobot_calon_dosen,
                        k=1)[0]
                    preferensi_hari_dosen = [d for d in pilihan_hari_dosen if d not in dosen_pengajar['preferensi']['hindari_hari']]
                    hari_dosen = random.choice(preferensi_hari_dosen)
                    
                    conflict = True
                    attempt = 1
                    excluded_day = []
                    excluded_room = []
                    while conflict and attempt <= max_attempt:
                        available_lecturer_schedule = find_available_schedule(jadwal_dosen, dosen_pengajar['nip'], hari_dosen)
                        available_room_schedule = find_available_schedule(jadwal_ruangan, ruang_dosen['kode'], hari_dosen)

                        available_room_x_lecturer = list(set(available_room_schedule) & set(available_lecturer_schedule))
                    
                        status = False
                        for jam in available_room_x_lecturer:
                            rentang_waktu = list(range(jam, jam + matkul['sks_akademik'] + 1))
                            if all(r in available_room_x_lecturer for r in rentang_waktu) and jam != 12:
                                status = True
                                conflict = False
                                jam_mulai_dosen = jam
                                jam_selesai_dosen = jam_mulai_dosen + matkul['sks_akademik']
                                break
                            else:
                                status = False
                            
                        if not status:
                            excluded_day.append(hari_dosen)
                            # print('pil ', pilihan_hari_dosen)
                            # print('exc ', excluded_day)
                            # print('      ', [d for d in pilihan_hari_dosen if d not in excluded_day])
                            # print('      ', available_room_x_lecturer)
                            if all(d in excluded_day for d in preferensi_hari_dosen):
                                excluded_room.append(ruang_dosen['kode'])
                                calon_ruang_pengganti = [rd for rd in calon_ruang_dosen if rd['kode'] not in excluded_room]
                                bobot_calon_ruang_pengganti =[(len(set(r['plot']) & set(matkul['bidang']))*10 or 1)for r in calon_ruang_pengganti]
                                ruang_dosen = random.choices(
                                    population=calon_ruang_pengganti, 
                                    weights=bobot_calon_ruang_pengganti, 
                                    k=1)[0]
                            else:
                                hari_dosen = random.choice([d for d in preferensi_hari_dosen if d not in excluded_day])

                        if attempt == max_attempt:
                            jam_selesai_dosen = 19
                            jam_mulai_dosen = jam_selesai_dosen - matkul['sks_akademik']
                        attempt += 1
                    
                    jadwal_ruangan[ruang_dosen['kode']].append({'hari': hari_dosen, 'jam_mulai': jam_mulai_dosen, 'jam_selesai': jam_selesai_dosen})
                    jadwal_dosen[dosen_pengajar['nip']].append({'hari': hari_dosen, 'jam_mulai': jam_mulai_dosen, 'jam_selesai': jam_selesai_dosen})
                    
                    sesi_dosen = JadwalKuliah(
                        kapasitas    = ruang_dosen['kapasitas'],
                        kode_matkul  = sesi.kode_matkul[:-3],
                        kode_dosen   = dosen_pengajar['nip'],
                        sks_akademik = matkul['sks_akademik'],
                        kode_ruangan = ruang_dosen['kode'],
                        hari         = hari_dosen,
                        jam_mulai    = jam_mulai_dosen,
                        jam_selesai  = jam_selesai_dosen,
                        tipe_kelas   = matkul['tipe_kelas'],
                        program_studi= matkul['prodi']
                    )
                    jadwal.append(sesi_dosen)

    return jadwal

In [5]:
def generate_jadwal(matakuliah_list, dosen_list, ruang_list):
    jadwal = []
    pilihan_hari_dosen = ["SENIN", "SELASA", "RABU", "KAMIS", "JUMAT"]
    pilihan_hari_asisten: list[str] = copy.deepcopy(pilihan_hari_dosen)
    pilihan_hari_asisten.append("SABTU")

    jadwal_dosen = {}    # {nip: [ {hari, jam_mulai, jam_selesai}, ... ]}
    jadwal_ruangan = {}  # {kode_ruangan: [ {hari, jam_mulai, jam_selesai}, ... ]}
    beban_dosen = {} # {nip: beban_sks}

    for dt in dosen_list:
        beban_dosen[dt['nip']] = 0

    max_attempt = 10

    for matkul in matakuliah_list:
        # DOSEN (reminder: dosen tidak tetap belum tersentuh)
        dosen_prodi = [d for d in dosen_list if d["prodi"] == matkul['prodi']]
        # print(f"prodi {matkul['prodi']} dosen_prodi {dosen_prodi}")
        bobot_dosen_berdasarkan_pakar = [(len(set(d['kepakaran']) & set(matkul['bidang'])) or 0) for d in dosen_prodi]
        
        # RUANGAN
        ruangan_prodi = [r for r in ruang_list if (matkul['prodi'] in r['plot'] or 'GENERAL' in r['plot'])]
        ruangan_prodi_prioritas = [r for r in ruangan_prodi if r['kapasitas'] > 35 and (r['tipe_ruangan'] != 'RAPAT' if matkul.get('asistensi') else True)]
        bobot_ruangan_prodi = [(len(set(r['plot']) & set(matkul['bidang']))*10 or 1) + int(r['kapasitas']) + (100 if r['tipe_ruangan'] == matkul['tipe_kelas'] else 1) for r in ruangan_prodi_prioritas or ruangan_prodi]
        
        ruangan_prodi_prioritas_praktikum = [r for r in ruangan_prodi if not (r['kapasitas'] > max([r['kapasitas'] for r in ruangan_prodi if r['tipe_ruangan'] == "PRAKTIKUM"])) and r['tipe_ruangan'] != 'RAPAT']
        bobot_ruangan_prodi_prioritas_praktikum = [((len(set(r['plot']) & set(matkul['bidang']))*10 or 1) + int(r['kapasitas'])) * (100 if r['tipe_ruangan'] == matkul['tipe_kelas'] else 1) for r in ruangan_prodi_prioritas_praktikum if r['tipe_ruangan'] != 'RAPAT']
        
        jumlah_mahasiswa = matkul['jumlah_mahasiswa']
        index_kelas = 1
        
        while jumlah_mahasiswa > 0:
            dosen_prodi_available = [dp for dp in dosen_prodi if beban_dosen[dp['nip']] < 12]
            # print(f'dosen_prodi_available {dosen_prodi_available}')
            bobot_dosen_prodi_available = [len(set(d['kepakaran']) & set(matkul['bidang']))*10 for d in dosen_prodi_available]
                        
            if matkul.get('asistensi') and not matkul.get('integrated_class') and matkul.get('tipe_kelas_asistensi') == "PRAKTIKUM":
                ruang_dosen = random.choices(
                    population=ruangan_prodi_prioritas_praktikum, 
                    weights=bobot_ruangan_prodi_prioritas_praktikum, 
                    k=1)[0]
            else:
                ruang_dosen = random.choices(
                    population=ruangan_prodi_prioritas or ruangan_prodi, 
                    weights=bobot_ruangan_prodi, 
                    k=1)[0]
            
            if dosen_prodi_available:
                dosen = random.choices(
                    population=dosen_prodi_available, 
                    weights=bobot_dosen_prodi_available, 
                    k=1)[0]
            else:
                dosen = random.choices(
                    population=dosen_prodi,
                    weights=bobot_dosen_berdasarkan_pakar,
                    k=1)[0]

            if dosen['nip'] not in jadwal_dosen: jadwal_dosen[dosen['nip']] = []
            preferensi_hari = [hari for hari in pilihan_hari_dosen if hari not in dosen['preferensi']['hindari_hari']]
            hari_dosen = random.choice(preferensi_hari)

            sukses = False
            attempt = 0
            while not sukses and max_attempt >= attempt:
                if ruang_dosen['kode'] not in jadwal_ruangan: jadwal_ruangan[ruang_dosen['kode']] = []

                jadwal_dosen_sesuai_hari = [d for d in jadwal_dosen[dosen['nip']] if d['hari'] == hari_dosen] if jadwal_dosen[dosen['nip']] else []
                jadwal_ruangan_sesuai_hari = [d for d in jadwal_ruangan[ruang_dosen['kode']] if d['hari'] == hari_dosen] if jadwal_ruangan[ruang_dosen['kode']] else []

                if jadwal_ruangan_sesuai_hari:
                    list_jam_selesai_ruangan = [r['jam_selesai'] for r in jadwal_ruangan_sesuai_hari]
                    jam_terakhir_ruangan = max(list_jam_selesai_ruangan)
                    jam_mulai_dosen = jam_terakhir_ruangan if jam_terakhir_ruangan <= (18 - matkul['sks_akademik']) else 0
                else:
                    jam_mulai_dosen = 7

                if jam_mulai_dosen != 0:
                    jam_selesai_dosen = jam_mulai_dosen + matkul['sks_akademik']
                    sukses = True
                else:
                    hari_dosen = random.choice(preferensi_hari)

                # kalo jadwal dosen ganda, ubah hari
                for jd in jadwal_dosen_sesuai_hari:
                    if jd['hari'] == hari_dosen and (jd['jam_mulai'] < jam_mulai_dosen < jd['jam_selesai'] or jd['jam_mulai'] == jam_mulai_dosen):
                        hari_dosen = random.choice(preferensi_hari) if matkul['kode'][-3:] != '-AS' else random.choice(pilihan_hari_asisten)
                        sukses = False

                attempt += 1

            sesi_dosen = JadwalKuliah(
                kapasitas       = ruang_dosen['kapasitas'],
                kode_matkul     = f"{matkul['kode']}{angka_ke_huruf(index_kelas)}",
                kode_dosen      = dosen['nip'],
                sks_akademik    = matkul['sks_akademik'],
                kode_ruangan    = ruang_dosen['kode'],
                hari            = hari_dosen,
                jam_mulai       = jam_mulai_dosen,
                jam_selesai     = jam_selesai_dosen,
                tipe_kelas      = matkul['tipe_kelas'],
                program_studi   = matkul['prodi']
            )
            jadwal.append(sesi_dosen)

            beban_dosen[dosen['nip']] += matkul['sks_akademik']
            jadwal_dosen[dosen['nip']].append({'hari': hari_dosen, 'jam_mulai': jam_mulai_dosen, 'jam_selesai': jam_selesai_dosen})
            jadwal_ruangan[ruang_dosen['kode']].append({'hari': hari_dosen, 'jam_mulai': jam_mulai_dosen, 'jam_selesai': jam_selesai_dosen})
            jumlah_mahasiswa -= ruang_dosen['kapasitas']

            # Penentuan kelas asistensi yang tidak terintegrasi dengan kelas dosen
            if matkul.get('asistensi') and not matkul.get('integrated_class'):
                # print(f"{matkul['nama']:<50} praktikum: {matkul.get('asistensi'):<10} integrated: {matkul.get('integrated_class')}")
                # print(f"kapasitas ruangan dosen {ruang_dosen['kapasitas']}; {min(40, max(40, ruang_dosen['kapasitas']))}")
                ruang_prodi_asistensi = [r for r in ruangan_prodi_prioritas or ruangan_prodi if r['tipe_ruangan'] == matkul.get('tipe_kelas_asistensi') and r['kapasitas'] >= ruang_dosen['kapasitas'] and r['tipe_ruangan'] != "RAPAT"]
                bobot_ruang_asisten = [(len(set(r['plot']) & set(matkul['bidang']))*10 or 1) for r in ruang_prodi_asistensi]

                # print(f"{ruang_dosen['kapasitas']} available asistance {ruang_prodi_asistensi}")
                ruang_asisten = random.choices(
                    population=ruang_prodi_asistensi, 
                    weights=bobot_ruang_asisten, 
                    k=1)[0]
                
                hari_asisten = random.choice(pilihan_hari_asisten[pilihan_hari_dosen.index(hari_dosen):])
                sukses = False
                attempt = 0
                isolated_day = []
                # print(matkul['kode'], hari_dosen, jam_mulai_dosen, jam_selesai_dosen)
                while not sukses and max_attempt >= attempt:
                    if ruang_asisten['kode'] not in jadwal_ruangan: jadwal_ruangan[ruang_asisten['kode']] = []

                    jadwal_ruangan_sesuai_hari = [d for d in jadwal_ruangan[ruang_asisten['kode']] if d['hari'] == hari_asisten] if jadwal_ruangan[ruang_asisten['kode']] else []

                    if jadwal_ruangan_sesuai_hari:
                        list_jam_selesai_ruangan = [r['jam_selesai'] for r in jadwal_ruangan_sesuai_hari]
                        jam_terakhir_ruangan = max(list_jam_selesai_ruangan)
                        jam_mulai_asisten = jam_terakhir_ruangan if jam_terakhir_ruangan <= (18 - matkul['sks_akademik']) else 0
                    else:
                        jam_mulai_asisten = 7

                    # print(f"  {matkul['kode']}-AS {hari_asisten} {jam_mulai_asisten}")
                    if jam_mulai_asisten != 0 and not (
                        hari_asisten == hari_dosen and 
                        not (
                            jam_mulai_asisten >= jam_selesai_dosen and 
                            (jam_mulai_asisten + matkul['sks_akademik']) <= jam_mulai_dosen
                        )
                    ): # Kalau jam asisten sesuai + Tidak bertabrakan dgn kelas dosen
                        jam_selesai_asisten = jam_mulai_asisten + matkul['sks_akademik']
                        sukses = True
                    else:
                        if hari_asisten == hari_dosen and not (
                            jam_mulai_asisten >= jam_selesai_dosen and 
                            (jam_mulai_asisten + matkul['sks_akademik']) <= jam_mulai_dosen
                        ):
                            isolated_day.append(hari_asisten)
                        dosen_exclude_current_day = [day for day in pilihan_hari_asisten[pilihan_hari_dosen.index(hari_dosen):] if day not in isolated_day]
                        hari_exclude_current_day = [day for day in pilihan_hari_asisten if day not in isolated_day]
                        hari_asisten = random.choice(dosen_exclude_current_day or hari_exclude_current_day)

                    attempt += 1
                
                sesi_asisten = JadwalKuliah(
                    kapasitas    = ruang_asisten['kapasitas'],
                    kode_matkul  = f"{matkul['kode']}{angka_ke_huruf(index_kelas)}-AS",
                    kode_dosen   = "AS",
                    sks_akademik = matkul['sks_akademik'],
                    kode_ruangan = ruang_asisten['kode'],
                    hari         = hari_asisten,
                    jam_mulai    = jam_mulai_asisten,
                    jam_selesai  = jam_selesai_asisten,
                    tipe_kelas   = matkul['tipe_kelas_asistensi'],
                    program_studi= matkul['prodi']
                )
                jadwal.append(sesi_asisten)
                jadwal_ruangan[ruang_asisten['kode']].append({'hari': hari_asisten, 'jam_mulai': jam_mulai_asisten, 'jam_selesai': jam_selesai_asisten})
            
            index_kelas += 1

    # 🛠️ REPAIR sebelum return
    # jadwal = repair_jadwal(jadwal, matakuliah_list, dosen_list, ruang_list)
    return jadwal

def generate_populasi(matakuliah_list, dosen_list, ruang_list, ukuran_populasi):
    return [generate_jadwal(matakuliah_list, dosen_list, ruang_list) for _ in range(ukuran_populasi)]

In [6]:
export_jadwal_to_excel(generate_jadwal(matakuliah_list, dosen_list, ruang_list))

✅ File 'jadwal_kuliah.xlsx' berhasil dibuat.
Excel disimpan di: /home/kevinprasetio/course_scheduling/dao


/home/kevinprasetio/course_scheduling/dao/jadwal_kuliah.xlsx

In [8]:
def tampilkan_jadwal(jadwal):
    print(f"\n📅 Jadwal Mata Kuliah Terbaik ({len(jadwal)} kelas):\n")
    print(f"{'QTY':<5}{'MK':<57}{'Dosen':<20}{'Ruang':<8}{'SKS':<10}{'Hari':<10}{'Jam Mulai':<12}{'Jam Selesai':<12}{'Tipe':<10}{'Program Studi'}")
    print("-" * 60)
    # # urutkan berdasarkan kode_matkul, hari dan jam_mulai
    # for sesi in sorted(jadwal, key=lambda x: (x.kode_matkul, x.hari, x.jam_mulai)):
    #     print(f"{sesi.kode_matkul:<8}{sesi.kode_dosen:<15}{sesi.kode_ruangan:<8}{sesi.sks_akademik:<10}{sesi.hari:<10}{sesi.jam_mulai:<12}{sesi.jam_selesai:<12}{sesi.tipe_kelas}")

    # urutkan berdasarkan kode_kelas (biar keliatan jelas yang bentrok kelas mana)
    for sesi in sorted(jadwal, key=lambda x: (x.kode_matkul, x.hari, x.jam_mulai)):
    # for sesi in sorted(jadwal, key=lambda x: (x.kode_dosen, x.hari, x.jam_mulai)):
    # for sesi in sorted(jadwal, key=lambda x: (x.kode_ruangan, x.hari, x.jam_mulai)):
    # for sesi in sorted(jadwal, key=lambda x: (x.hari, x.jam_mulai)):
        # print(sesi.kode_matkul, sesi.kode_matkul[:-2], sesi.kode_matkul[:-5])
        nama_matkul = next((m['nama'] for m in matakuliah_list if m['kode'] == sesi.kode_matkul[:-1] or m['kode'] == sesi.kode_matkul[:-4]), '')
        nama_dosen = next((d['nama'] for d in dosen_list if d['nip'] == sesi.kode_dosen), sesi.kode_dosen) if sesi.kode_dosen != "AS" else "ASISTEN"
        print(f"{sesi.kapasitas:<5}{sesi.kode_matkul:<12}{nama_matkul:<45}{sesi.kode_dosen:<6}{nama_dosen:<14}{sesi.kode_ruangan:<8}{sesi.sks_akademik:<10}{sesi.hari:<10}{sesi.jam_mulai:<12}{sesi.jam_selesai:<12}{sesi.tipe_kelas:<10}{sesi.program_studi}")

In [ ]:
# ==================================== TESTING POPULASI + TAMPILKAN ===========================================

def cetak(jadwal):
    for sesi in jadwal:
            print(sesi)        
        
populasi = generate_populasi(matakuliah_list, dosen_list, ruang_list, 1)
for j in populasi:
    tampilkan_jadwal(j)

In [9]:
'''
    Tambahkan pengecekan fitness:
    - 
'''

def hitung_fitness(jadwal): # tambahkan parameter ruang_list, dosen_list, matakuliah
    penalti = 0
    jadwal_dosen = {}
    jadwal_ruangan = {}
    beban_dosen = {}

    # COUNTER
    hitung_dosen_bentrok = 0
    hitung_ruangan_bentrok = 0
    hitung_asdos_nabrak_dosen = 0
    hitung_kelas_dosen_missing = 0

    kapasitas_mata_kuliah = {}

    for sesi in jadwal:
        info_ruangan = next((r for r in ruang_list if r["kode"] == sesi.kode_ruangan), None)

        if sesi.kode_dosen != "AS":
            if sesi.kode_matkul[:-1] not in kapasitas_mata_kuliah: kapasitas_mata_kuliah[sesi.kode_matkul[:-1]] = 0
            kapasitas_mata_kuliah[sesi.kode_matkul[:-1]] += info_ruangan['kapasitas']
            
            info_dosen = next((d for d in dosen_list if d["nip"] == sesi.kode_dosen), {})
            preferensi = info_dosen.get("preferensi", {})
            
            # Cek Beban SKS Dosen
            beban_dosen[sesi.kode_dosen] = beban_dosen.get(sesi.kode_dosen, 0) + sesi.sks_akademik
            if beban_dosen[sesi.kode_dosen] > 12:
                penalti += BOBOT_PENALTI['dosen_terbebani']

            # Cek Dosen tidak boleh ganda
            if sesi.kode_dosen not in jadwal_dosen:
                jadwal_dosen[sesi.kode_dosen] = [] # bikin space kalo belum ada data berkaitan

            for sesi_lain in jadwal_dosen[sesi.kode_dosen]:
                if sesi.hari == sesi_lain['hari']:
                    if not (sesi.jam_mulai >= sesi_lain['jam_selesai'] or sesi.jam_selesai <= sesi_lain['jam_mulai']):
                        penalti += BOBOT_PENALTI['dosen_bentrok']
                        hitung_dosen_bentrok += 1
            
            jadwal_dosen[sesi.kode_dosen].append({'hari': sesi.hari, 'jam_mulai': sesi.jam_mulai, 'jam_selesai': sesi.jam_selesai})

            # Cek Dosen tidak boleh Sabtu
            if sesi.hari == "Sabtu":
                penalti += BOBOT_PENALTI['weekend_class']

            # Cek Preferensi dosen
            if sesi.hari in preferensi.get("hindari_hari", []):
                penalti += BOBOT_PENALTI['melanggar_preferensi']
            for waktu in preferensi.get("hindari_jam", []):
                if waktu in range(sesi.jam_mulai, sesi.jam_selesai):
                    penalti += BOBOT_PENALTI['melanggar_preferensi']
                    break
        else:
            sesi_dosen = next((j for j in jadwal if j.kode_matkul == sesi.kode_matkul[:-3]), None)
            if sesi_dosen:
                if sesi.hari == sesi_dosen.hari and not (sesi.jam_mulai >= sesi_dosen.jam_selesai or sesi.jam_selesai <= sesi_dosen.jam_mulai):
                    # print(f"    bentrok dosen {sesi_dosen.kode_matkul} {sesi_dosen.kode_ruangan} {sesi_dosen.hari} {sesi_dosen.jam_mulai} - asdos {sesi.kode_matkul} {sesi.kode_ruangan} {sesi.hari} {sesi.jam_mulai}")
                    penalti += BOBOT_PENALTI['asdos_nabrak_dosen']
                    hitung_asdos_nabrak_dosen += 1
            else:
                penalti += BOBOT_PENALTI["kelas_dosen_tidak_ditemukan"]
                hitung_kelas_dosen_missing += 1
            
        # Cek Ruangan tidak boleh digunakan lebih dari 1 matkul secara bersamaan
        if sesi.kode_ruangan not in jadwal_ruangan:
            jadwal_ruangan[sesi.kode_ruangan] = [] # bikin space kalo belum ada data berkaitan

        for sesi_lain in jadwal_ruangan[sesi.kode_ruangan]:
            if sesi.hari == sesi_lain['hari']:
                if not (sesi.jam_mulai >= sesi_lain['jam_selesai'] or sesi.jam_selesai <= sesi_lain['jam_mulai']):
                    penalti += BOBOT_PENALTI['ruangan_bentrok']
                    hitung_ruangan_bentrok += 1

        jadwal_ruangan[sesi.kode_ruangan].append({'hari': sesi.hari, 'jam_mulai': sesi.jam_mulai, 'jam_selesai': sesi.jam_selesai})

        # Cek Salah tipe ruangan
        if sesi.tipe_kelas != info_ruangan['tipe_ruangan']:
            penalti += BOBOT_PENALTI['salah_tipe_ruangan']

    # Cek Apakah kapasitas total matkul bisa mencakup semua mahasiswa aktif
    for kode_matkul, kapasitas in kapasitas_mata_kuliah.items():
        data_matkul = next((matkul for matkul in matakuliah_list if matkul['kode'] == kode_matkul), None)
        if data_matkul:
            if kapasitas < data_matkul['jumlah_mahasiswa']:
                penalti += BOBOT_PENALTI['kapasitas_kelas_terbatas']

    # if beban_dosen: print(f"{'':<10}{'beban dosen':<40}: {beban_dosen}")
    # if hitung_dosen_bentrok: print(f"{'':<10}{'Bentrok Dosen':<40} : {hitung_dosen_bentrok}")
    # if hitung_ruangan_bentrok: print(f"{'':<10}{'Bentrok Ruangan':<40} : {hitung_ruangan_bentrok}")
    # if hitung_asdos_nabrak_dosen: print(f"{'':<10}{'Bentrok Dosen-Asdos':<40} : {hitung_asdos_nabrak_dosen}")
    # if hitung_kelas_dosen_missing: print(f"{'':<10}{'Kelas Dosen Missing':<40} : {hitung_kelas_dosen_missing}")
    # if any([hitung_asdos_nabrak_dosen, hitung_dosen_bentrok, hitung_ruangan_bentrok]): print()
    # print(f"{'final fitness':<50} : {max(0, 1000 - penalti)}")
    # return max(0, 1000 - penalti)
    return 1000-penalti

In [ ]:
# ==================================== TESTING HITUNG FITNESS ===========================================
x = 1
max_loop = 1
minimum = []
maximum = []
while x <= max_loop:
    jadwal = generate_populasi(matakuliah_list, dosen_list, ruang_list, 20)
    skor_fitness = [hitung_fitness(j) for j in jadwal]
    print(f"{'default':<15}min: {min(skor_fitness)}, max: {max(skor_fitness)}")
    minimum.append(min(skor_fitness))
    maximum.append(max(skor_fitness))
    x += 1

print(f"{'average minimum':<20}: {sum(minimum) / len(minimum):<10}, {'minimum of all time':<20}: {min(minimum)}")
print(f"{'average maximum':<20}: {sum(maximum) / len(maximum):<10}, {'maximum of all time':<20}: {max(maximum)}")

jadwal_repaired = [repair_jadwal(j, matakuliah_list, dosen_list, ruang_list) for j in jadwal]
new_skor_fitness = [hitung_fitness(j) for j in jadwal_repaired]
print(f"{'1st repair':<15}min: {min(new_skor_fitness)}, max: {max(new_skor_fitness)}")

# jadwal_repaired = [repair_jadwal(j, matakuliah_list, dosen_list, ruang_list) for j in jadwal]
# new_skor_fitness = [hitung_fitness(j) for j in jadwal_repaired]
# print(f"{'2nd repair':<15}min: {min(new_skor_fitness)}, max: {max(new_skor_fitness)}")

# jadwal_repaired = [repair_jadwal(j, matakuliah_list, dosen_list, ruang_list) for j in jadwal]
# new_skor_fitness = [hitung_fitness(j) for j in jadwal_repaired]
# print(f"{'3rd repair':<15}min: {min(new_skor_fitness)}, max: {max(new_skor_fitness)}")

In [10]:
def roulette_selection(populasi, fitness_scores):
    """
    Mencari waktu yang tersedia dari jadwal ruangan berdasarkan hari yang dipilih.

    Args:
        jadwal_ruangan (list): List jadwal ruangan yang sudah ada.
        kode_ruangan (str): Kode ruangan yang akan diperiksa.
        hari (str): Hari yang akan diperiksa jadwalnya.

    Returns:
        list: List jadwal yang bisa digunakan (jam).
    """
    
    total_fitness = sum(fitness_scores)
    pick = random.uniform(0, total_fitness)
    current = 0
    for individu, score in zip(populasi, fitness_scores):
        current += score
        if current > pick:
            return individu
    return random.choice(populasi)  # fallback, jika tidak ketemu

In [11]:
def crossover(parent1, parent2):
    """
    Mencari waktu yang tersedia dari jadwal ruangan berdasarkan hari yang dipilih.

    Args:
        jadwal_ruangan (list): List jadwal ruangan yang sudah ada.
        kode_ruangan (str): Kode ruangan yang akan diperiksa.
        hari (str): Hari yang akan diperiksa jadwalnya.

    Returns:
        list: List jadwal yang bisa digunakan (jam).
    """
    
    titik = random.randint(1, len(parent1)-1)
    child1 = parent1[:titik] + parent2[titik:]
    child2 = parent2[:titik] + parent1[titik:]
    return child1, child2

In [12]:
def mutasi(individu, dosen_list, ruang_list, peluang_mutasi=0.1):
    """
    Mencari waktu yang tersedia dari jadwal ruangan berdasarkan hari yang dipilih.

    Args:
        jadwal_ruangan (list): List jadwal ruangan yang sudah ada.
        kode_ruangan (str): Kode ruangan yang akan diperiksa.
        hari (str): Hari yang akan diperiksa jadwalnya.

    Returns:
        list: List jadwal yang bisa digunakan (jam).
    """
    
    pilihan_hari = ["SENIN", "SELASA", "RABU", "KAMIS", "JUMAT", "SABTU"]
    pilihan_waktu = list(range(7, 19))

    for sesi in individu:
        if random.random() < peluang_mutasi:
            # Randomly mutate hari, jam, atau ruangan
            attr = random.choice(['hari', 'jam', 'ruang'])

            if attr == 'hari':
                sesi.hari = random.choice(pilihan_hari)
            elif attr == 'jam':
                jam_mulai = random.choice(pilihan_waktu)
                sesi.jam_mulai = jam_mulai
                sesi.jam_selesai = jam_mulai + sesi.sks_akademik
            elif attr == 'ruang':
                ruang = random.choice(ruang_list)
                sesi.kode_ruangan = ruang['kode']
    return individu

In [13]:
'''
    KEJADIAN ERROR:
    - kelas asisten kehilangan kelas dosen SETELAH proses CROSSOVER (mengakibatkan error di perhitungan fitness) 
        Perkiraan Solusi: Perbaiki jadwal dengan tambahkan pengecekan pasangan kelas dosen dan asisten
'''

def genetic_algorithm(matakuliah_list, dosen_list, ruang_list, ukuran_populasi=30, jumlah_generasi=100, peluang_mutasi=0.1, proporsi_elite=0.05):
    avg_per_gen = []
    
    populasi = generate_populasi(matakuliah_list, dosen_list, ruang_list, ukuran_populasi)
    populasi = [repair_jadwal(j, matakuliah_list, dosen_list, ruang_list) for j in populasi]

    best_fitness_global = float('-inf')
    best_individual_global = None

    jumlah_elite = max(1, int(ukuran_populasi * proporsi_elite))

    for gen in range(jumlah_generasi):
        fitness_scores = [hitung_fitness(individu) for individu in populasi]

        # Simpan individu terbaik
        individu_elite = [individu for _, individu in sorted(zip(fitness_scores, populasi), key=lambda x: x[0], reverse=True)][:jumlah_elite]

        next_gen = individu_elite.copy()

        # Generate anak baru
        while len(next_gen) < ukuran_populasi:
            parent1 = roulette_selection(populasi, fitness_scores)
            parent2 = roulette_selection(populasi, fitness_scores)

            child1, child2 = crossover(parent1, parent2)

            child1 = mutasi(child1, dosen_list, ruang_list, peluang_mutasi)
            child2 = mutasi(child2, dosen_list, ruang_list, peluang_mutasi)

            child1 = repair_jadwal(child1, matakuliah_list, dosen_list, ruang_list)
            child2 = repair_jadwal(child2, matakuliah_list, dosen_list, ruang_list)

            next_gen.append(child1)
            if len(next_gen) < ukuran_populasi:
                next_gen.append(child2)

        populasi = next_gen

        fitness_scores = [hitung_fitness(individu) for individu in populasi]
        gen_best_fitness = max(fitness_scores)
        gen_best_individual = populasi[fitness_scores.index(gen_best_fitness)]

        if gen_best_fitness > best_fitness_global:
            best_fitness_global = gen_best_fitness
            best_individual_global = copy.deepcopy(gen_best_individual)

        # REMINDER: HAPUS SECTION INI KALAU LESAI DEV
        best_fitness_global = gen_best_fitness
        best_individual_global = copy.deepcopy(gen_best_individual)
        avg = sum(fitness_scores) / len(fitness_scores)
        avg_per_gen.append(avg)
        
        print(f"{f'[Gen {gen}]':<15}Fitness Terbaik: {gen_best_fitness}")
        print(f"{'':<5}AVG : {avg}")
        print(f"{'':<5}BEST: {best_fitness_global}")

    print(f"\nOverall Fitness Average: {sum(avg_per_gen) / len(avg_per_gen)}")
    return best_individual_global

In [ ]:
# generate_populasi(matakuliah_list, dosen_list, ruang_list, 100)
# best_schedule = genetic_algorithm(matakuliah_list, dosen_list, ruang_list)
best_schedule = genetic_algorithm(
    matakuliah_list, dosen_list, ruang_list, 
    ukuran_populasi=75, jumlah_generasi=250, peluang_mutasi=0.05
)
# print('\n')
export_jadwal_to_excel(best_schedule)
# tampilkan_jadwal(best_schedule)